# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: gNB-CU experiences segmentation fault upon receiving UE Context Modification Response with mutated DU ID
Repro Steps:
1) The attacker intercepts a legitimate UE Context Modification request.
2) Modify the RNTI in the IE to an incorrect value and forward it to the gNB-CU.
3) For gNB-DUs, the attacker waits for UE release and immediately sends a DLRRCMessage Transfer.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "gNB-CU segfaults when processing UE Context Modification Response carrying a tampered/mismatched DU identifier; repro manipulates RNTI/IDs and may trigger DLRRCMessageTransfer after UE release.",
    "interface_guess": ["F1AP", "RRC"],
    "procedure_guess": ["UE Context Modification", "DL RRC Message Transfer", "UE Context Release"],
    "components_involved": ["gNB-CU", "gNB-DU", "UE"],
    "key_ids": {
      "transaction_id": "",
      "cu_ue_f1ap_id": "",
      "du_ue_f1ap_id": "",
      "du_id": "mutated",
      "rnti": "mutated",
      "pci": "",
      "served_cell_list": []
    },
    "signals_or_timers": [
      "UEContextModificationRequest",
      "UEContextModificationResponse",
      "DLRRCMessageTransfer",
      "UEContextRelease",
      "C-RNTI"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": ["UE Context Modification", "Response", "IDs", "validation"],
      "title": ["§8.3.4.2 UE CONTEXT MODIFICATION RESPONSE – Successful Operation"]
    },
    {
      "Keywords": ["DL RRC Message Transfer", "CU->DU", "old DU ID", "context"],
      "title": ["§8.4.2 DL RRC Message Transfer", "§9.2.3.2 DL RRC MESSAGE TRANSFER"]
    },
    {
      "Keywords": ["gNB-CU UE F1AP ID", "identifier", "scope"],
      "title": ["§9.3.1.4 gNB-CU UE F1AP ID"]
    },
    {
      "Keywords": ["gNB-DU UE F1AP ID", "identifier", "scope"],
      "title": ["§9.3.1.5 gNB-DU UE F1AP ID"]
    },
    {
      "Keywords": ["C-RNTI", "allocation", "in response"],
      "title": ["§9.3.1.32 C-RNTI"]
    }
  ],
  "expected_behaviour": "UE Context Modification Response shall include both gNB-CU UE F1AP ID and gNB-DU UE F1AP ID; the CU must bind them to an existing UE-associated logical F1-connection and ignore/reject on mismatch. If present, C-RNTI indicates DU allocation for this UE context. DL RRC MESSAGE TRANSFER shall carry valid CU/DU UE F1AP IDs (optionally an old DU ID) and must not be accepted for a released/nonexistent context without proper establishment; no crash should occur."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "F1AP/f1ap_cu_ue_context_management.c",
      "function_name": "CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE",
      "reason": "CU entry point handling UE Context Modification Response; site of crash when IDs are malformed/mismatched."
    },
    {
      "path": "F1AP/lib/f1ap_ue_context.c",
      "function_name": "decode_ue_context_mod_resp",
      "reason": "Parses and validates mandatory gNB-CU and gNB-DU UE F1AP IDs from Response."
    },
    {
      "path": "F1AP/f1ap_handlers.c",
      "function_name": "f1ap_messages_processing matrix",
      "reason": "Routes UEContextModification successful outcome to CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE."
    },
    {
      "path": "F1AP/lib/f1ap_rrc_message_transfer.c",
      "function_name": "decode_dl_rrc_message_transfer",
      "reason": "Requires valid CU/DU UE F1AP IDs; relevant to second repro (DLRRCMessageTransfer after release)."
    },
    {
      "path": "F1AP/f1ap_cu_rrc_message_transfer.c",
      "function_name": "CU_send_DL_RRC_MESSAGE_TRANSFER",
      "reason": "Shows CU-side construction of DL RRC MESSAGE TRANSFER with IDs for context binding."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§8.4.2.2 DL RRC Message Transfer – Successful operation",
      "text": "The gNB-CU initiates the procedure by sending a DL RRC MESSAGE TRANSFER message. If a UE-associated logical F1-connection exists, the DL RRC MESSAGE TRANSFER message shall contain the gNB-DU UE F1AP ID IE... The message shall include, if available, the old gNB-DU UE F1AP ID IE so that the gNB-DU can retrieve the existing UE context." 
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§9.2.3.2 DL RRC MESSAGE TRANSFER",
      "text": "Mandatory IEs include: gNB-CU UE F1AP ID, gNB-DU UE F1AP ID, SRB ID, RRC-Container. The old gNB-DU UE F1AP ID is optional." 
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§9.3.1.4 gNB-CU UE F1AP ID; §9.3.1.5 gNB-DU UE F1AP ID",
      "text": "The gNB-CU UE F1AP ID uniquely identifies the UE association over F1 within the gNB-CU. The gNB-DU UE F1AP ID uniquely identifies the UE association over F1 within the gNB-DU." 
    },
    {
      "kind": "spec",
      "source": "F1AP.docx",
      "location": "§8.3.4.2 UE CONTEXT MODIFICATION RESPONSE – Successful Operation",
      "text": "If the C-RNTI IE is included in the UE CONTEXT MODIFICATION RESPONSE, the gNB-CU shall consider that the C-RNTI has been allocated by the gNB-DU for this UE context." 
    },
    {
      "kind": "code",
      "source": "F1AP/lib/f1ap_ue_context.c",
      "location": "1999-2013",
      "text": "F1AP_LIB_FIND_IE(... id_gNB_CU_UE_F1AP_ID, true); F1AP_LIB_FIND_IE(... id_gNB_DU_UE_F1AP_ID, true); ... case id_gNB_CU_UE_F1AP_ID: out->gNB_CU_ue_id = ...; case id_gNB_DU_UE_F1AP_ID: out->gNB_DU_ue_id = ...;" 
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_cu_ue_context_management.c",
      "location": "155-166",
      "text": "if (!decode_ue_context_mod_resp(pdu, &resp)) { LOG_E(F1AP, \"cannot decode F1 UE Context Modification Response\"); ... } ... F1AP_UE_CONTEXT_MODIFICATION_RESP(msg_p) = resp;" 
    },
    {
      "kind": "code",
      "source": "F1AP/lib/f1ap_rrc_message_transfer.c",
      "location": "399-406",
      "text": "F1AP_LIB_FIND_IE(... id_gNB_CU_UE_F1AP_ID, true); out->gNB_CU_ue_id = ...; F1AP_LIB_FIND_IE(... id_gNB_DU_UE_F1AP_ID, true); out->gNB_DU_ue_id = ...;" 
    },
    {
      "kind": "code",
      "source": "F1AP/f1ap_handlers.c",
      "location": "53-61",
      "text": "{DU_handle_UE_CONTEXT_SETUP_REQUEST, CU_handle_UE_CONTEXT_SETUP_RESPONSE, 0}, ... {DU_handle_UE_CONTEXT_MODIFICATION_REQUEST, CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE, 0}, ... {DU_handle_DL_RRC_MESSAGE_TRANSFER, 0, 0}," 
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UEContextModificationRequest", "precond": "UE context established", "postcond": "DU processes request"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UEContextModificationResponse (with CU/DU UE F1AP IDs, optional C-RNTI)", "precond": "Request accepted", "postcond": "CU validates IDs"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UEContextReleaseCommand (if mismatch/cleanup)", "precond": "ID mismatch or UE release criteria", "postcond": "DU releases context"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "DLRRCMessageTransfer (optional old DU ID)", "precond": "UE-associated logical F1-connection exists", "postcond": "DU delivers RRC"}
  ],
  "state_machine": {
    "states": ["Idle", "ContextEstablished", "ModReqSent", "AwaitingModResp", "ContextReleased", "AbnormalMismatch"],
    "transitions": [
      {"from": "Idle", "to": "ContextEstablished", "on": "UE Context Setup Completed"},
      {"from": "ContextEstablished", "to": "ModReqSent", "on": "Send UEContextModificationRequest"},
      {"from": "ModReqSent", "to": "AwaitingModResp", "on": "Await Response"},
      {"from": "AwaitingModResp", "to": "ContextEstablished", "on": "Receive Valid Response", "guard": "CU/DU IDs match existing binding"},
      {"from": "AwaitingModResp", "to": "AbnormalMismatch", "on": "Receive Response", "guard": "ID mismatch/unknown UE"},
      {"from": "AbnormalMismatch", "to": "ContextReleased", "on": "Send UEContextReleaseCommand"},
      {"from": "ContextReleased", "to": "ContextEstablished", "on": "New association established", "guard": "DLRRCMessageTransfer with valid IDs triggers (re)establishment per spec"}
    ]
  },
  "assumptions": [
    "Segfault likely due to missing null checks or unchecked ID lookup on Response path.",
    "Attacker can mutate DU UE F1AP ID or RNTI in transit causing context lookup failure.",
    "DLRRCMessageTransfer after release without valid IDs should be rejected, not crash."
  ],
  "questions": [
    "Exact stack trace location of the segfault?",
    "Were IDs or C-RNTI fields malformed or simply mismatched?",
    "Is there any path that accepts DLRRCMessageTransfer when context is released?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "UEContextModification Response must include CU/DU UE F1AP IDs",
      "result": "PASS",
      "evidence": ["spec §8.3.4.2", "code F1AP/lib/f1ap_ue_context.c:1999-2013"]
    },
    {
      "name": "DLRRCMessageTransfer direction CU->DU with valid IDs",
      "result": "PASS",
      "evidence": ["spec §8.4.2; §9.2.3.2", "code F1AP/lib/f1ap_rrc_message_transfer.c:399-406", "code F1AP/f1ap_handlers.c:60-61"]
    },
    {
      "name": "ID binding: IDs map to existing UE-associated F1-connection before use",
      "result": "UNKNOWN",
      "evidence": ["spec §8.4.2.2 (context lookup semantics)", "code F1AP/f1ap_cu_ue_context_management.c:155-166 (no explicit binding check visible)"]
    },
    {
      "name": "Robust handling on ID mismatch (no crash)",
      "result": "FAIL",
      "evidence": ["Observed segfault in CU when DU ID tampered", "spec mandates ignore/reject behavior, not crash"]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "Specs require presence of CU/DU UE F1AP IDs and context-bound processing; code decodes IDs but crash on malformed/mismatched IDs indicates insufficient validation/guarding in CU response path, consistent with an implementation defect rather than spec nonconformance."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "F1AP.docx §8.4.2.2 DL RRC Message Transfer – Successful operation",
      "F1AP.docx §9.2.3.2 DL RRC MESSAGE TRANSFER",
      "F1AP.docx §9.3.1.4 gNB-CU UE F1AP ID",
      "F1AP.docx §9.3.1.5 gNB-DU UE F1AP ID",
      "F1AP.docx §9.3.1.32 C-RNTI",
      "F1AP.docx §8.3.4.2 UE CONTEXT MODIFICATION RESPONSE – Successful Operation"
    ],
    "code_citations": [
      "F1AP/lib/f1ap_ue_context.c:1999-2013",
      "F1AP/f1ap_cu_ue_context_management.c:155-166",
      "F1AP/lib/f1ap_rrc_message_transfer.c:399-406",
      "F1AP/f1ap_handlers.c:55-61"
    ],
    "expected_vs_observed": "Expected: CU validates CU/DU UE F1AP IDs from UE Context Modification Response and, on mismatch or unknown context, rejects/ignores without crash; DL RRC MESSAGE TRANSFER is accepted only for valid/established contexts. Observed: Tampered DU ID (and/or RNTI) leads to CU segmentation fault when handling the Response; sending DLRRCMessageTransfer after release risks processing without proper context, indicating missing guards."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "Spec requires CU to use CU/DU UE F1AP IDs to bind to an existing UE-associated F1 context and to ignore/reject malformed/mismatched inputs. The CU decodes IDs but appears to dereference/use data before verifying binding, leading to a segfault on tampered DU ID (and possibly race with release).",
    "citations": {
      "spec": [
        "F1AP.docx §8.4.2.2",
        "F1AP.docx §9.2.3.2",
        "F1AP.docx §9.3.1.4",
        "F1AP.docx §9.3.1.5",
        "F1AP.docx §9.3.1.32",
        "F1AP.docx §8.3.4.2"
      ],
      "code": [
        "F1AP/lib/f1ap_ue_context.c:1999-2013",
        "F1AP/f1ap_cu_ue_context_management.c:155-166",
        "F1AP/lib/f1ap_rrc_message_transfer.c:399-406",
        "F1AP/f1ap_handlers.c:55-61"
      ]
    },
    "suggested_repro": [
      "Establish UE context (normal attach).",
      "Send UEContextModificationRequest (CU->DU).",
      "Tamper the gNB-DU UE F1AP ID (or C-RNTI) in-flight and deliver UEContextModificationResponse to CU.",
      "Observe CU process in CU_handle_UE_CONTEXT_MODIFICATION_RESPONSE and downstream path.",
      "Release UE, then attempt DL RRC MESSAGE TRANSFER with invalid/old IDs to ensure rejection without crash."
    ],
    "risks": [
      "Remote DoS of gNB-CU via crafted F1AP messages.",
      "Potential undefined behavior beyond segfault (memory corruption)."
    ],
    "next_steps": [
      "Add binding checks in CU handling path (map CU/DU IDs to UE context); return error on miss.",
      "Harden decoders and handlers with null checks and bounds validation before ITTI dispatch.",
      "Add regression tests covering ID mismatch and post-release DL RRC handling.",
      "Audit adjacent procedures for similar unchecked ID paths."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.